In [1]:
import pandas as pd

In [2]:
a=pd.read_csv("iso3_to_wb_name.csv").rename(columns={"country":"wb_name"})

In [15]:
b=pd.read_csv("country_codes_extra.csv")
b.columns= b.columns.str.lower()

In [18]:
r=pd.merge(a,b,on = 'iso3')
r.head()

,iso3,wb_name,short name,official name,iso2,uni,undp,faostat,gaul
0,AFG,Afghanistan,Afghanistan,The Islamic Republic of Afghanistan,AF,4,AFG,2,1
1,AGO,Angola,Angola,the Republic of Angola,AO,24,ANG,7,8
2,ALB,Albania,Albania,the Republic of Albania,AL,8,ALB,3,3
3,AND,Andorra,Andorra,the Principality of Andorra,AD,20,AND,6,7
4,ARE,United Arab Emirates,United Arab Emirates,the United Arab Emirates,AE,784,UAE,225,255


In [19]:
r.to_csv("iso3_WBname_iso2_uni_undp_fasotat_gaul.csv",index=False)